# А/Б тест.

### Цель исследования
провести оценку результатов A/B-теста - изменений, связанных с внедрением улучшенной рекомендательной системы.

Оценить происходит ли улучшение следующих метрик:

конверсии в просмотр карточек товаров — событие product_page,\
просмотры корзины — product_cart,\
покупки — purchase.

### Порядок исследования
перед нами 4 датасета:

ab_project_marketing_events.csv - календарь маркетинговых событий на 2020 год, в котором отражена информация о всех маркетинговых мероприятиях;\
final_ab_new_users.csv - информация о пользователях, зарегистрировавшиеся с 7 по 21 декабря 2020 года;\
final_ab_events.csv - действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года;\
final_ab_participants.csv - таблица участников тестов.

О качестве данных ничего не известно, на первоначальном этапе требуется изучить датасет и выполнить предобработку данных,\
выполнив проверку на соответствие наименований столбцов, наличие дубликатов, пропусков, корректность типов данных,\
уникальность значений, убедиться в полноте данных.

После выполнения предобработки необходимо оценить корректность проведения теста и обратить вниание на:
- соответствие данных требованиям технического задания. Проверить корректность всех пунктов технического задания.
- время проведения теста. Убедиться, что оно не совпадает с маркетинговыми и другими активностями.
- аудиторию теста. Удостовериться, что нет пересечений с конкурирующим тестом и нет пользователей, участвующих в двух группах теста одновременно.\
Проверить равномерность распределения по тестовым группам и правильность их формирования.


Провести исследовательский анализ данных и ответить на следующие вопросы:
- Количество событий на пользователя одинаково распределены в выборках?
- Как число событий в выборках распределено по дням?
- Как меняется конверсия в воронке в выборках на разных этапах?
- Какие особенности данных нужно учесть, прежде чем приступать к A/B-тестированию?

Оценить результаты A/B-тестирования и ответить на вопросы:
- Что можно сказать про результаты A/В-тестирования?
- Проверить статистическую разницу долей z-критерием.

И описать выводы по этапу исследовательского анализа данных и по проведённой оценке результатов A/B-тестирования.\
Сформулировать общее заключение о корректности проведения теста.

### Техническое задание
Название теста: recommender_system_test;\
группы: А — контрольная, B — новая платёжная воронка;дата запуска: 2020-12-07;\
дата остановки набора новых пользователей: 2020-12-21;\
дата остановки: 2021-01-04;\
аудитория: 15% новых пользователей из региона EU;\
назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;\
ожидаемое количество участников теста: 6000.\
ожидаемый эффект: за 14 дней с момента регистрации пользователи покажут улучшение каждой метрики не менее, чем на 10%:\
конверсии в просмотр карточек товаров — событие product_page,\
просмотры корзины — product_cart,
покупки — purchase.

### Описание данных

ab_project_marketing_events.csv — календарь маркетинговых событий на 2020 год.

Структура файла:

name — название маркетингового события;\
regions — регионы, в которых будет проводиться рекламная кампания;\
start_dt — дата начала кампании;\
finish_dt — дата завершения кампании.


final_ab_new_users.csv — пользователи, зарегистрировавшиеся с 7 по 21 декабря 2020 года.

Структура файла:

user_id — идентификатор пользователя;\
first_date — дата регистрации;\
region — регион пользователя;
device — устройство, с которого происходила регистрация.


final_ab_events.csv — действия новых пользователей в период с 7 декабря 2020 по 4 января 2021 года.

Структура файла:

user_id — идентификатор пользователя;\
event_dt — дата и время покупки;\
event_name — тип события;\
details — дополнительные данные о событии.\
Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.


final_ab_participants.csv — таблица участников тестов.

Структура файла:

user_id — идентификатор пользователя;\
ab_test — название теста;\
group — группа пользователя.

## Загрузка и предобработка данных.

In [19]:
#загрузим нужные библиотеки
import pandas as pd
import numpy as np
from numpy import mean
import warnings
import math as mth
import datetime as dt
import seaborn as sns
sns.set_style("dark")
import scipy.stats as st
import plotly.express as px
import matplotlib.pyplot as plt
from plotly import graph_objects as go
from datetime import datetime, timedelta
warnings.simplefilter("ignore") 
pd.set_option('display.float_format', '{:,.2f}'.format)

In [20]:
#загрузим данные
try:
   marketing_events, new_users, act_users, participants = (
    pd.read_csv("C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14.1_АВ_тест\\файлы\\ab_project_marketing_events.csv", sep=','), #маркетинговые мероприятия
    pd.read_csv("C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14.1_АВ_тест\\файлы\\final_ab_new_users.csv", sep=','), #новые пользователи
    pd.read_csv("C:\\Users\\kirio\\OneDrive\\Рабочий стол\\ЯПрактикум\\14.1_АВ_тест\\файлы\\final_ab_events.csv", sep=','),#действия пользователей
    pd.read_csv("C:\\Users\\kirio\OneDrive\\Рабочий стол\\ЯПрактикум\\14.1_АВ_тест\\файлы\\final_ab_participants.csv", sep=',')#участники теста
)
except:
   marketing_events, new_users, act_users, participants = (
    pd.read_csv('https://code.s3.yandex.net/datasets/ab_project_marketing_events.csv', sep=','), #маркетинговые мероприятия
    pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_new_users.csv', sep=','), #новые пользователи
    pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_events.csv', sep=','),#действия пользователей
    pd.read_csv('https://code.s3.yandex.net/datasets/final_ab_participants.csv', sep=',')#участники теста
)

### Изучение датасетов

In [21]:
# создадим функцию get_info для вывода информации о датасетах
def get_info (data):
    data.info(memory_usage='deep')
    display(data.head())
    display(data.describe())
    print('\033[1m' + 'Количество дубликатов' + '\033[1m',data.duplicated().sum())

In [22]:
get_info(marketing_events)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 4.0 KB


,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


,name,regions,start_dt,finish_dt
count,14,14,14,14
unique,14,6,14,14
top,Single's Day Gift Promo,APAC,2020-06-25,2020-07-11
freq,1,4,1,1


Количество дубликатов 0


In [23]:
#изменим тип данных на формат даты
marketing_events['start_dt']=pd.to_datetime(marketing_events['start_dt'])
marketing_events['finish_dt']=pd.to_datetime(marketing_events['finish_dt'])

In [24]:
#все маркетинговые мероприятия
marketing_events.head(14)

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11
5,Black Friday Ads Campaign,"EU, CIS, APAC, N.America",2020-11-26,2020-12-01
6,Chinese New Year Promo,APAC,2020-01-25,2020-02-07
7,Labor day (May 1st) Ads Campaign,"EU, CIS, APAC",2020-05-01,2020-05-03
8,International Women's Day Promo,"EU, CIS, APAC",2020-03-08,2020-03-10
9,Victory Day CIS (May 9th) Event,CIS,2020-05-09,2020-05-11


In [25]:
get_info(new_users)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 15.4 MB


,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


,user_id,first_date,region,device
count,61733,61733,61733,61733
unique,61733,17,4,4
top,3EE718562551ECDD,2020-12-21,EU,Android
freq,1,6290,46270,27520


Количество дубликатов 0


In [26]:
# изменим тип данных на формат даты
new_users['first_date']=pd.to_datetime(new_users['first_date'])

Уникальные значения region и device:

In [27]:
new_users['region'].value_counts()

EU           46270
N.America     9155
CIS           3155
APAC          3153
Name: region, dtype: int64

In [28]:
new_users['device'].value_counts()

Android    27520
PC         15599
iPhone     12530
Mac         6084
Name: device, dtype: int64

In [29]:
get_info(act_users)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 93.4 MB


,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


,details
count,"62,740.00"
mean,23.88
std,72.18
min,4.99
25%,4.99
50%,4.99
75%,9.99
max,499.99


Количество дубликатов 0


В таблице с действиями пользователей есть пропущенные значения в details. Дубликатов нет.

Изменим тип данных на формат даты.

Изучим пропущенные значения. Предположим details заполнены только для покупок.

In [30]:
#сгруппируем по названию  события и поверим
act_users.pivot_table(index='event_name', values='details', aggfunc='count')

,details
event_name,
login,0
product_cart,0
product_page,0
purchase,62740


В столбце details отражена стоимость товара для  purchase, по остальным этапам воронки  пропущенные значения можно заменить на 0


In [33]:
act_users['details']=act_users['details'].fillna(0)

In [34]:
#изменитм тип
act_users['event_dt']=pd.to_datetime(act_users['event_dt'])

In [35]:
get_info(participants)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 3.6 MB


,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


,user_id,group,ab_test
count,18268,18268,18268
unique,16666,2,2
top,D0533ECF2A67B35C,A,interface_eu_test
freq,2,9655,11567


Количество дубликатов 0


Пропущенных значений нет. Формат данных задан верно.\
Проверим значания в колонке ab_test.

In [36]:
participants['ab_test'].value_counts()

interface_eu_test          11567
recommender_system_test     6701
Name: ab_test, dtype: int64

### Вывод:

Выполнена предобработка данных, изменены типы данных на формат даты. Дубликаты не обнаружены.

Количество пользователей проживающих в следующих регионах:

EU           - 46270\
N.America    - 9155\
CIS          - 3155\
APAC         - 3153


Девайсы, количество:

Android    - 27520\
PC         - 15599\
iPhone     - 12530\
Mac        - 6084

В таблице act_users обнаружены пропущенные значения по колонке details. Пропуски были по всем этапам воронки продаж, кроме purchase.\
Это связано с тем, что в поле details фиксируется стоимость покупки на этапе оплаты товара. Все пропуски были заменены на 0.

Воронка событий состоит из следующих ступеней:

login (авторизация пользователя);
product_page (страница с выбором продуктов);
product_cart (корзина продуктов);
purchase (покупка).

В таблице participants отражена информация о принадлежности пользователя к группе теста.\

Тестирование проводится по 2 группам:

interface_eu_test           - 11567\
recommender_system_test     - 6701
